# Mask R-CNN laoding

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

# Root directory of the project
ROOT_DIR = os.path.abspath("./maskrcnn/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "OTB50/Car2")
gt_dir=IMAGE_DIR+"/groundtruth_rect.txt"
img_dir=IMAGE_DIR+"/img/"

#Configurations
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
#config.display()
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

Using TensorFlow backend.


## Run Object Detection

In [ ]:
from math import sqrt
if  not (os.path.isfile('goturn.caffemodel') and os.path.isfile('goturn.prototxt')):
    errorMsg = '''
    Could not find GOTURN model in current directory.
    Please ensure goturn.caffemodel and goturn.prototxt are in the current directory
    '''

    print(errorMsg)
    sys.exit()

frame_index=1    
resize_scale=2
    
    

# Create tracker
tracker = cv2.TrackerGOTURN_create()   
tracker2 = cv2.TrackerGOTURN_create()   

# print(gt_dir)
# print(img_dir)
file = open(gt_dir, 'r') 

for line in file: 
    
    
    print(frame_index)
    image_name=img_dir+str(frame_index).zfill(4)+".jpg"
    print(image_name)
    #Load an color image in grayscale
    img = cv2.imread(image_name,1)
    img2 = cv2.imread(image_name,1)
    gt_img=cv2.imread(image_name,1)
    mask_img=cv2.imread(image_name,1)
    gotunrn_img=cv2.imread(image_name,1)
    gotunrn_img2=cv2.imread(image_name,1)
    print(line[1])
    print(img.shape)
    height,width,channel=img.shape
    line=line.split(",")
    x1=int(line[0])
    y1=int(line[1])
    x2=int(line[2])
    y2=int(line[3])
    print(x1)
    print(x2)
    print(y1)
    print(y2)
    
    if(frame_index==1):
        bbox = (x1,y1, x2, y2)
        bbox2 = (x1,y1, x2, y2)
        gotunrn_img = tracker.init(img,bbox)
        gotunrn_img2 = tracker2.init(img2,bbox2)
        frame_index+=1
        continue
    
    gotunrn_img, bbox = tracker.update(img)
    gotunrn_img2, bbox2 = tracker2.update(img2)
      # Draw bounding box
    if gotunrn_img:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cx1 =int(bbox[0] + bbox[2]/2)
        cy1 =int(bbox[1] + bbox[3]/2)
        cw1=bbox[2]
        ch1=bbox[3]
        cv2.rectangle(img, p1, p2, (255,255,0), 2, 1)
        #cv2.circle(img, (cx1,cy1), 3,(255,255,0), 1, 8,0)
#         print("==================goturn=================")
#         print(cx1)
#         print(cy1)
#         print(cw1)
#         print(ch1)
#         print("==================goturn=================")
    else :
        # Tracking failure
        print("tracking1 fail")
    # Display result
#     cv2.imshow("Tracking", img) 
    if gotunrn_img2:
        # Tracking success
        p3 = (int(bbox2[0]), int(bbox2[1]))
        p4 = (int(bbox2[0] + bbox2[2]), int(bbox2[1] + bbox2[3]))
        cx2 =int(bbox2[0] + bbox2[2]/2)
        cy2 =int(bbox2[1] + bbox2[3]/2)
        cw2=bbox2[2]
        ch2=bbox2[3]
        cv2.rectangle(img2, p3, p4, (0,0,255), 2, 1)
        #cv2.circle(img, (cx2,cy2), 3,(255,255,255), 1, 8,0)
#         print("==================goturn+maskrcnn=================")
#         print(cx2)
#         print(cy2)
#         print(cw2)
#         print(ch2)
#         print("==================goturn+maskrcnn=================")
    else :
        # Tracking failure
        print("tracking1 fail")
    # Display result
    cv2.imshow("new goturn", img2) 
    cv2.imshow("goturn", img) 
    resized_image = cv2.resize(mask_img, (width*resize_scale, height*resize_scale)) 
    image=resized_image
    results = model.detect([image], verbose=1)
    r = results[0]
    range_in_r=len(r["rois"])
    
    for i in range(range_in_r):
        p1=(r["rois"][i][1],r["rois"][i][0])
        p2=(r["rois"][i][3],r["rois"][i][2])
        cx3=int(r["rois"][i][1]+(r["rois"][i][3]-r["rois"][i][1])/2)
        cy3=int(r["rois"][i][0]+(r["rois"][i][2]-r["rois"][i][0])/2)
        cw3=int(r["rois"][i][3]-r["rois"][i][1])
        ch3=int(r["rois"][i][2]-r["rois"][i][0])
        
        dist = sqrt( (cx3/resize_scale - cx2)**2 + (cy3/resize_scale - cy2)**2 )
        cv2.rectangle(image, p1, p2, (255,0,0), 2, 1) 
        if(dist<=5and dist<5):
            tmp_id=class_names[r["class_ids"][i]]
            tmp_score=r['scores'][i]
            tmp_final=str(tmp_id)+str(tmp_score)
            font = cv2.FONT_HERSHEY_SIMPLEX
            
            #cv2.circle(image, (cx3,cy3), 3,(255,0,0), 1, 8,0)
            print("==================maskrcnn=================")
            print(cx3/resize_scale)
            print(cy3/resize_scale)
            print(cw3)
            print(ch3)
            print(dist)
            widthDiff=abs(cx3/resize_scale -cx2)
            heightDiff=abs(cx3/resize_scale -cx2)
            
            print("-------differetn---")
            print(widthDiff)
            print(heightDiff)
            print(cx3/resize_scale/20)
            print(cy3/resize_scale/20)
            if(widthDiff>=3 or heightDiff>=3 or dist>3):
                print("updategoturn +++++++++++++++++++++++++++++++++")
                update_x=r["rois"][i][1]/resize_scale
                update_y=r["rois"][i][0]/resize_scale
                update_w=cw3/resize_scale
                update_h=ch3/resize_scale
                print(p1)
                print(p2)

                print("update")
           
                print(update_x)
                print(update_y)
                print(update_w)
                print(update_h)
                print("gt")
                print(x1)
                print(y1)
                print(x1+x2)
                print(y1+y2)
                bbox2 = (update_x,update_y, update_w, update_h)
                tracker2 = cv2.TrackerGOTURN_create()  
                gotunrn_img2 = tracker2.init(img2,bbox2)
            print("==================maskrcnn=================")
            cv2.putText(image,tmp_final,p1, font, 1,(255,255,255),2,cv2.LINE_AA)
    resized_back_image = cv2.resize(image, (width, height)) 
    cv2.imshow('maskrcnn',resized_back_image)
    #cv2.imshow('orgin',img)
    cv2.rectangle(gt_img, (x1,y1), (x1+x2,y1+y2), (0,255,0), 3, 1) 
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(gt_img,"gt",(x1,y1+5), font, 0.5,(255,255,255),1,cv2.LINE_AA)
    cv2.imshow('gt_img',gt_img)
   
    cv2.waitKey(1)
    
    frame_index+=1
        # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break
    if(frame_index>500):
        break
cv2.destroyAllWindows()

1
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0001.jpg
6
(240, 320, 3)
76
64
79
52
2
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0002.jpg
6
(240, 320, 3)
76
64
79
52
Processing 1 images
image                    shape: (480, 640, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
3
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0003.jpg
6
(240, 320, 3)
76
63
79
52
Processing 1 images
image                    shape: (480, 640, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 

==================maskrcnn=================
105.0
103.0
132
106
2.23606797749979
-------differetn---
1.0
1.0
5.25
5.15
==================maskrcnn=================
16
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0016.jpg
5
(240, 320, 3)
75
62
78
51
Processing 1 images
image                    shape: (480, 640, 3)         min:   46.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
104.5
101.5
133
103
1.5811388300841898
-------differetn---
0.5
0.5
5.225
5.075
==================maskrcnn=================
17
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0017.jpg
5
(240, 320, 3)
75
62
78
51
Processing 1 images
image                    shape: (480, 

==================maskrcnn=================
105.0
102.0
131
103
2.23606797749979
-------differetn---
1.0
1.0
5.25
5.1
==================maskrcnn=================
28
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0028.jpg
5
(240, 320, 3)
75
62
78
51
Processing 1 images
image                    shape: (480, 640, 3)         min:   46.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
104.5
102.5
133
104
0.7071067811865476
-------differetn---
0.5
0.5
5.225
5.125
==================maskrcnn=================
29
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0029.jpg
4
(240, 320, 3)
74
62
78
51
Processing 1 images
image                    shape: (480, 6

40
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0040.jpg
5
(240, 320, 3)
75
62
78
51
Processing 1 images
image                    shape: (480, 640, 3)         min:   42.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
105.5
103.0
127
99
2.5
-------differetn---
2.5
2.5
5.275
5.15
==================maskrcnn=================
41
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0041.jpg
5
(240, 320, 3)
75
62
78
51
Processing 1 images
image                    shape: (480, 640, 3)         min:   41.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              sha

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
104.5
102.0
129
102
3.640054944640259
-------differetn---
3.5
3.5
5.225
5.1
updategoturn +++++++++++++++++++++++++++++++++
(145, 153)
(274, 255)
update
72.5
76.5
64.5
51.0
gt
75
78
136
129
==================maskrcnn=================
53
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0053.jpg
5
(240, 320, 3)
75
61
78
50
Processing 1 images
image                    shape: (480, 640, 3)         min:   44.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                 

...

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
99.5
97.5
115
93
3.8078865529319543
-------differetn---
3.5
3.5
4.975
4.875
updategoturn +++++++++++++++++++++++++++++++++
(142, 149)
(257, 242)
update
71.0
74.5
57.5
46.5
gt
72
75
128
122
==================maskrcnn=================
124
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0124.jpg
2
(240, 320, 3)
72
56
76
46
Processing 1 images
image                    shape: (480, 640, 3)         min:   44.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                

==================maskrcnn=================
100.0
101.5
117
96
2.0615528128088303
-------differetn---
2.0
2.0
5.0
5.075
==================maskrcnn=================
136
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0136.jpg
3
(240, 320, 3)
73
56
80
47
Processing 1 images
image                    shape: (480, 640, 3)         min:   43.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
100.0
102.5
119
97
4.031128874149275
-------differetn---
4.0
4.0
5.0
5.125
updategoturn +++++++++++++++++++++++++++++++++
(141, 157)
(260, 254)
update
70.5
78.5
59.5
48.5
gt
73
80
129
127
==================maskrcnn=================
137
/home/ou-lab/Desktop/cap5415Finalproject/maskrcn

149
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0149.jpg
4
(240, 320, 3)
74
56
81
46
Processing 1 images
image                    shape: (480, 640, 3)         min:   43.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
102.5
104.0
122
97
2.5
-------differetn---
2.5
2.5
5.125
5.2
==================maskrcnn=================
150
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0150.jpg
4
(240, 320, 3)
74
56
81
46
Processing 1 images
image                    shape: (480, 640, 3)         min:   43.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              sh

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
102.0
95.5
119
95
1.8027756377319946
-------differetn---
1.0
1.0
5.1
4.775
==================maskrcnn=================
162
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0162.jpg
5
(240, 320, 3)
75
56
73
47
Processing 1 images
image                    shape: (480, 640, 3)         min:   43.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn===============

==================maskrcnn=================
103.5
97.5
123
98
2.5495097567963922
-------differetn---
2.5
2.5
5.175
4.875
==================maskrcnn=================
174
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0174.jpg
6
(240, 320, 3)
76
56
75
46
Processing 1 images
image                    shape: (480, 640, 3)         min:   40.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
==================maskrcnn=================
102.5
97.0
123
99
1.8027756377319946
-------differetn---
1.5
1.5
5.125
4.85
==================maskrcnn=================
175
/home/ou-lab/Desktop/cap5415Finalproject/maskrcnn/OTB50/Car2/img/0175.jpg
5
(240, 320, 3)
75
56
75
46
Processing 1 images
image                    shape: (480,

print(gt_dir)